In [5]:
import os

from Crypto.Cipher import Salsa20
from Crypto.Hash import HMAC, SHA256
from Crypto.Random import get_random_bytes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

def passwd():
    backend = default_backend()
    salt = os.urandom(16)
    # PBKDF2 derivation function
    kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=64,
    salt=salt,
    iterations=100000,
    backend=backend
    )
    # pass-phrase
    password = b"Some password"
    # Encrypt password with PBKDF2
    key = kdf.derive(password)
    # Return key and salt
    return key, salt
    
def encrypt():
    plainfile = open("file", 'rb')
    key, salt = passwd()
    with open("salt", 'wb') as saltfile:
        saltfile.write(salt)
    keySalsa20 = key[0:32]
    keySha256 = key[32:]
    cipher = Salsa20.new(key=keySalsa20)
    msg = cipher.nonce + cipher.encrypt(plainfile.read())
    h = HMAC.new(keySha256, msg, digestmod=SHA256)
    hex = h.hexdigest()
    with open("Salsa20", 'wb') as salsafile:
        salsafile.write(msg)
    with open("Sha256", 'w') as shafile:
        shafile.write(hex)
        
encrypt()

In [6]:
import os

from Crypto.Cipher import Salsa20
from Crypto.Hash import HMAC, SHA256
from Crypto.Random import get_random_bytes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

def passwd():
    backend = default_backend()
    saltfile = open("salt", 'rb')
    salt = saltfile.read()
    # PBKDF2 derivation function
    kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=64,
    salt=salt,
    iterations=100000,
    backend=backend
    )
    # pass-phrase
    password = b"Some password"
    # Encrypt password with PBKDF2
    key = kdf.derive(password)
    # Get keys for Salsa20 and Sha256
    secretSalsa20 = key[0:32]
    secretSha256 = key[32:]
    # Return the secrets
    return secretSalsa20, secretSha256
    
def decrypt():
    secretSalsa20, secretSha256 = passwd()
    with open("Salsa20", 'rb') as salsafile:
        h = HMAC.new(secretSha256, salsafile.read(), digestmod=SHA256)
    try:
        with open("Sha256", 'r') as shafile:
            h.hexverify(shafile.read())
        with open("Salsa20", 'rb') as salsafile:
            msg_nonce = salsafile.read(8)
            ciphertext = salsafile.read()
        cipher = Salsa20.new(key=secretSalsa20, nonce=msg_nonce)
        text = cipher.decrypt(ciphertext)
        with open('decrypted', 'wb') as decryptedfile:
            decryptedfile.write(text)
    except ValueError:
        print("Message has been compromised")
        
decrypt()